In [ ]:
pip install fpdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=68cb5f298b6281321f09fde7f8b33ffdce4492689eab228cca6d4c41ac08c3c1
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
pip show fpdf

Name: fpdf
Version: 1.7.2
Summary: Simple PDF generation for Python
Home-page: http://code.google.com/p/pyfpdf
Author: Olivier PLATHEY ported by Max
Author-email: maxpat78@yahoo.it
License: LGPLv3+
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [ ]:
pip install telebot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=46b312bbff33a67d71f27825ba7d5855761ee0244f6804246ad28d44c66cac86
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [ ]:
pip install gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install telebot_calendar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for telebot_calendar: filename=telebot_calendar-1.2-py3-none-any.whl size=5829 sha256=73767f23e86737a5e6d91675169a7cb20260bddab8214fc057e0a989d3b552f1
  Stored in directory: /root/.cache/pip/wheels/1e/85/64/133440cba45b9b84dca050e046bf605cfcb63f701d48e95a69
Successfully built telebot_calendar


In [ ]:
import telebot
from telebot import types
import fpdf
from fpdf import FPDF
from random import randint
from datetime import datetime
import pandas as pd
import yaml
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import email
import email.mime.application
import mimetypes
import gspread   # для работы с гугл шитс
from google.oauth2.service_account import Credentials
import datetime 
from telebot_calendar import Calendar, CallbackData, RUSSIAN_LANGUAGE 
from telebot.types import ReplyKeyboardRemove, CallbackQuery 
from telebot.types import ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton 


users = {}  # id: fio
places = {}  # id: place
studak_tickets = {}
ind_fio = {}  # авторизация фио
ind_tick = {}  # авторизация студак
dates_consa = {}
questions_consa = {}

bot = telebot.TeleBot('6075532965:AAFVHMtcfKSs_nF_k-zrV0-X1aCJQurA6Fo')

calendar = Calendar(language=RUSSIAN_LANGUAGE) 
calendar_1_callback = CallbackData("calendar_1", "action", "year", "month", "day") 

markup = types.InlineKeyboardMarkup(row_width=1)
case1 = types.InlineKeyboardButton(text="Справка об обучении",callback_data='Обучение')
case2 = types.InlineKeyboardButton(text="Справка о болезни",callback_data='Болезнь')
case3 = types.InlineKeyboardButton(text="Запись на консультацию",callback_data='Конса')
case4 = types.InlineKeyboardButton(text="Пересдачи",callback_data='Пересдача')
case5 = types.InlineKeyboardButton(text="Важная информация",url = 'http://www.iate.obninsk.ru/node/51', callback_data='Инфа')
case6 = types.InlineKeyboardButton(text="Контакты",callback_data='Контакты')
markup.add(case1, case2, case3, case4, case5, case6)

scopes = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file('bot-sha2-94a5230fc5f9.json',scopes=scopes)
gc = gspread.authorize(credentials)

sh = gc.open("Студ офис бд")


def get_data_values():
  worksheet = sh.worksheet('база')
  data = worksheet.get_all_values()
  data = pd.DataFrame(data[1:], columns = data[0])
  return data
data = get_data_values()   # получение базы данных

# отправка данных о запросе на справку об обучении на почту

def send_mail(id, place):
  mail_content = f"<p><b>Запрос на предоставление справки об обучении</b>.</p>Имя студента: {users[id]}</p>Наименование места предоставления: {place}"   # код авторизации, который высылатся на почту
  sender_address = 'test_account14@mail.ru'
  sender_pass = 'bieTtnJTnipAyViNi7Mv'
  message = MIMEMultipart()    # эксземпляр класса
  message['From'] = sender_address    # адрес отправителя
  message['To'] = sender_address    # адрес получателя
  message['Subject'] = 'Справка об обучении'
  message.attach(MIMEText(mail_content, 'html'))    # отправка кода авторизации

  session = smtplib.SMTP_SSL("smtp.mail.ru:465")
  session.login(sender_address, sender_pass)     # авторизация
  text = message.as_string()
  session.sendmail(sender_address, sender_address, text)   # отправка соо
  session.quit()


# переселыка на почту справки о болезни формата пдф

def sendmail_file(id, filename):

  msg = MIMEMultipart()
  msg['Subject'] = 'Справка об отсутствии/болезни'
  msg['From'] = 'test_account14@mail.ru'
  msg['To'] = 'test_account14@mail.ru'

  mail_content = f"""<p><b>Справка об отсутсвии/болезни студента.</b></p>ФИО: {users[id]}."""
  msg.attach(MIMEText(mail_content, 'html')) 

  fp=open(filename,'rb')
  att = email.mime.application.MIMEApplication(fp.read(),_subtype="pdf")
  fp.close()
  att.add_header('Content-Disposition','attachment',filename=filename)
  msg.attach(att)

  s = smtplib.SMTP('smtp.mail.ru')
  s.starttls()
  s.login('test_account14@mail.ru','bieTtnJTnipAyViNi7Mv')
  s.sendmail('test_account14@mail.ru','test_account14@mail.ru', msg.as_string())
  s.quit()



@bot.message_handler(commands=['start'])
def start(message):
  if message.chat.id not in users.keys():
    print(message.chat.id)
  bot.send_message(message.from_user.id, f"Добро пожаловать в чат-бота Студенческого офиса!\nДля авторизации напиши своё *ФИО (полностью)*", parse_mode = 'Markdown')
  bot.register_next_step_handler(message, get_fio) #следующий шаг – функция get_name
  

@bot.message_handler(commands=['back'])
def back(message):
    bot.send_message(message.from_user.id, f"Выберите услугу из предложенных ниже:",reply_markup = markup)
  

@bot.message_handler(content_types=['text'])
def get_fio(message): #получаем фио
    global fio
    fio = message.text
    if fio in data['ФИО'].values:
      users[message.chat.id] = fio
      ind_fio[id] = data.index [data['ФИО']== fio ]. tolist ()[0]
      bot.send_message(message.from_user.id, 'Введите номер своего Студенческого билета')
      bot.register_next_step_handler(message, get_studak)
      print(users)
    else:
      bot.send_message(message.from_user.id, 'Вы не обнаружены в реестре студентов. Повторите попытку ввода ФИО')
      bot.register_next_step_handler(message, get_fio)

def get_studak(message):
    global studak
    studak = message.text
    if studak in data['студенческий билет'].values:
      studak_tickets[message.chat.id] = studak
      ind_tick[id] = data.index [data['студенческий билет']== studak ]. tolist ()[0]
      if ind_tick[id] == ind_fio[id]:
        bot.send_message(message.from_user.id, "Авторизация пройдена, теперь ты можешь выбрать услугу\n\n_Чтобы возвращаться к данному меню вводите /back_",  reply_markup = markup, parse_mode='Markdown')
      else:
        bot.send_message(message.from_user.id, "Вы не обнаружены в реестре студентов. Повторите попытку ввода номера Студенческого билета")
        bot.register_next_step_handler(message, get_studak)
    else:
      bot.send_message(message.from_user.id, "Вы не обнаружены в реестре студентов. Повторите попытку ввода номера Студенческого билета")
      bot.register_next_step_handler(message, get_studak)



def check_other_messages(message): 
      questions_consa[message.chat.id] = message.text
      print(questions_consa)

      now = datetime.datetime.now()  # Get the
      # current date 
      bot.send_message( 
          message.chat.id, 
          "Выбор даты консультации", 
          reply_markup=calendar.create_calendar( 
              name=calendar_1_callback.prefix, 
              year=now.year, 
              month=now.month,  # Specify the NAME of your calendar 
          ), 
      ) 
 
@bot.callback_query_handler( 
    func=lambda call: call.data.startswith(calendar_1_callback.prefix) 
) 
def callback_inline(call: CallbackQuery): 
 
      name, action, year, month, day = call.data.split(calendar_1_callback.sep) 
      # Processing the calendar. Get either the date or None if the buttons are of a different type 
      date = calendar.calendar_query_handler( 
          bot=bot, call=call, name=name, action=action, year=year, month=month, day=day 
      ) 
      # There are additional steps. Let's say if the date DAY is selected, you can execute your code. I sent a message. 
      if action == "DAY": 
          bot.send_message( 
              chat_id=call.from_user.id, 
              text=f"Консультации проходят с 12:00 до 14:00, Вы выбрали дату {date.strftime('%d.%m.%Y')}", 
              reply_markup=ReplyKeyboardRemove(), 
          ) 
          date = date.strftime('%d.%m.%Y')
          dates_consa[call.message.chat.id] = date
          # print(f"{calendar_1_callback}: Day: {date}") 
          print(dates_consa)

          # def zapis_consa():
          id = call.from_user.id
          worksheet =  sh.worksheet('консультации')
          values_fio = worksheet.col_values(1) # 1 столбец - все ФИО

          if users[id] in values_fio:
            cell = worksheet.find(users[id]) 
            bot.send_message(call.message.chat.id, f'Вы уже записаны на консультацию {worksheet.cell(cell.row, 2).value}; {worksheet.cell(cell.row, 3).value}.\nНо вы можете её отменить.')
            bot.send_message(call.message.chat.id, "Чтобы вернуться к основному меню, введите /back")
          else:
            worksheet.update_cell(len(values_fio)+1,1, users[id])
            worksheet.update_cell(len(values_fio)+1,2, dates_consa[id])
            worksheet.update_cell(len(values_fio)+1,3, questions_consa[id])
            bot.send_message(call.message.chat.id, f'Вы успешно записаны на консультацию.')
            bot.send_message(call.message.chat.id, "Чтобы вернуться к основному меню, введите /back")

        
      elif action == "CANCEL": 
          bot.send_message( 
              chat_id=call.from_user.id, 
              text="", 
              reply_markup=ReplyKeyboardRemove(), 
          ) 
          print(f"{calendar_1_callback}: Cancellation") 

      


def get_spravka_offline(message):
    global place
    place = message.text
    send_mail(message.chat.id, place)
    bot.send_message(message.chat.id, "Запрос на получение справки отправлен.\n\nСправка будет готова в течение 3-х рабочих дней\nЕё можно будет получить по адресу: ул. Уличная\nВремя работы (будние дни): 9:00-18:00")
    bot.send_message(message.chat.id, "Чтобы вернуться к основному меню, введите /back")
   

def get_spravka_online(message):
        global place
        place = message.text
        places[message.chat.id] = place
        pdf = FPDF()
        pdf.add_page()   # создание страницы

        pdf.add_font('DejaVu', '','DejaVuSansCondensed.ttf', uni=True )   # шрифт
        pdf.set_font('DejaVu', size = 14)   # размер шрифта

        date = datetime.datetime.today().strftime('%d.%m.%y')
        number = randint(11111,99999)
        text1 = date+' № ' + str(number)
        pdf.cell(200,  # ширина клетки
                10,  # высота,
                txt = text1,
                ln = 1, # разрыв строки
                align = 'L'  # по центру
                )

        pdf.ln(h=5)
        pdf.set_line_width(5)
        pdf.cell(200,  # ширина клетки
                10,  # высота,
                txt = 'СПРАВКА',
                ln = 1, # разрыв строки
                align = 'C'  # по центру
                )
        pdf.ln(h=3)
        pdf.cell(200,  # ширина клетки
                10,  # высота,
                txt = f'{users[message.chat.id]}',
                ln = 1, # разрыв строки
                align = 'C'  # по центру
                )

        # pdf.cell(200,  # ширина клетки
        #         10,  # высота,
        #         txt = f'Дата рождения: ',
        #         ln = 1, # разрыв строки
        #         align = 'C'  # по центру
        #         )
        pdf.ln(h=3)
        kurs = data[data["ФИО"].values==users[message.chat.id]]['курс'].values[0]
        pdf.multi_cell(200,
                      10,
                      txt = f'обучается на {kurs} курсе по очной форме обучения по основной образовательной программе высшего образования - программе бакалавриата по направлению подготовки "Прикладная информатика"',
                      align = 'C'  # по центру
                      )

        pdf.cell(200,  # ширина клетки
                10,  # высота,
                txt = f'Финансовый Университет при Правительстве РФ.',
                ln = 1, # разрыв строки
                align = 'C'  # по центру
                )
        pdf.multi_cell(200,  # ширина клетки
                10,  # высота,
                txt = f'Для предоставления в «{places[message.chat.id]}»',
                align = 'C'  # по центру
                )
        pdf.ln(h=10)
        pdf.cell(200,  # ширина клетки
                10,  # высота,
                txt = f'Руководитель службы «Студенческий офис»:',
                ln = 1, # разрыв строки
                align = 'L'  # по центру
                )
        pdf.cell(200,  # ширина клетки
                10,  # высота,
                txt = f'Студофисов С.О.',
                ln = 1, # разрыв строки
                align = 'L'  # по центру
                )

        img = '/content/Bez-nazvaniya.png'
        pdf.image(img,  x=150, y=115,h=30,w=50 )
        pechat = '/content/f761f.jpg'
        pdf.image(pechat,  x=135, y=118,h=20,w=20 )
        result = pdf.output(f'Справка об обучении ({users[message.chat.id]}).pdf')
        bot.send_message(message.chat.id, "Ваша справка:")
        bot.send_document(message.chat.id, document=open(f'Справка об обучении ({users[message.chat.id]}).pdf', 'rb'))
        bot.send_message(message.chat.id, "Чтобы вернуться к основному меню, введите /back")

def get_ill_date_start(message):
  format = "%d-%m-%Y"
  res = True
  try:
    res = bool(datetime.datetime.strptime(str(message.text), format))
  except ValueError:
    res = False
  if res == True:
    if message.text != '/back':
      bot.register_next_step_handler(message, get_ill_date_finish)
      bot.send_message(message.from_user.id, 'Введи дату окончания болезни. Формат: dd-mm-yyyy')
    else:
      bot.send_message(message.from_user.id, f"Выберите услугу из предложенных ниже:",reply_markup = markup)

  elif res == False:
    if message.text != '/back':
      bot.send_message(message.from_user.id, 'Обрати внимание на формат: dd-mm-yyyy')
      bot.register_next_step_handler(message, get_ill_date_start)
    else:
      bot.send_message(message.from_user.id, f"Выберите услугу из предложенных ниже:",reply_markup = markup)


def get_ill_date_finish(message):
  format = "%d-%m-%Y"
  res = True
  try:
    res = bool(datetime.datetime.strptime(str(message.text), format))
  except ValueError:
    res = False
  if res == True:
    if message.text != '/back':
      bot.send_message(message.from_user.id, 'Вышли справку о болезни в pdf формате (документ/скан)')
      bot.register_next_step_handler(message, spravka_uploaded)
    else:
      bot.send_message(message.from_user.id, f"Выберите услугу из предложенных ниже:",reply_markup = markup)

  if res == False:
    if message.text != '/back':
      bot.send_message(message.from_user.id, 'Обрати внимание на формат: dd-mm-yyyy')
      bot.register_next_step_handler(message, get_ill_date_finish)
    else:
      bot.send_message(message.from_user.id, f"Выберите услугу из предложенных ниже:",reply_markup = markup)





@bot.message_handler(content_types=['document'])
def spravka_uploaded(message):
  try:
    chat_id = message.chat.id

    file_info = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    src = message.document.file_name
    with open(src, 'wb') as new_file:
        new_file.write(downloaded_file)
    sendmail_file(chat_id, src)
    bot.reply_to(message, "Ваша справка успешно принята для обработки.")
    bot.send_message(message.chat.id, "Чтобы вернуться к основному меню, введите /back")
  except Exception as e:
    if message.text != '/back':
      bot.send_message(message.chat.id, 'Неправильный формат справки.\nПовторите попытку')
      bot.register_next_step_handler(message, spravka_uploaded)
    else:
      bot.send_message(message.from_user.id, f"Выберите услугу из предложенных ниже:",reply_markup = markup)


@bot.callback_query_handler(func=lambda call: True)   # обработка клавиатуры
def query_handler(call):
    bot.answer_callback_query(callback_query_id=call.id)
    answer = ''
    if call.data == 'Обучение':
        markup_educ = types.InlineKeyboardMarkup(row_width=1)
        case1 = types.InlineKeyboardButton(text="Получить онлайн",callback_data='Онлайн')
        case2 = types.InlineKeyboardButton(text="Получить очно",callback_data='Очно')
        markup_educ.add(case1, case2)
        bot.send_message(call.message.chat.id, "Выберите способ получения справки:",  reply_markup = markup_educ)

    elif call.data == 'Болезнь':
        bot.send_message(call.message.chat.id, "Введи дату начала болезни. Формат: dd-mm-yyyy")
        bot.register_next_step_handler(call.message, get_ill_date_start)

    elif call.data == 'Конса':
        markup_consa = types.InlineKeyboardMarkup(row_width=1)
        case1 = types.InlineKeyboardButton(text="Записаться на консультацию",callback_data='Запись')
        case2 = types.InlineKeyboardButton(text="Отменить запись",callback_data='Отмена')
        markup_consa.add(case1, case2)
        bot.send_message(call.message.chat.id, "Выберите дейcтвие:",  reply_markup = markup_consa)
                # bot.register_next_step_handler(call.message, get_consa)

    elif call.data == 'Пересдача':
        markup_per = types.InlineKeyboardMarkup(row_width=1)
        case1 = types.InlineKeyboardButton(text="Уважительная причина",callback_data='Уваж')
        case2 = types.InlineKeyboardButton(text="Неуважительная причина",callback_data='Неуваж')
        markup_per.add(case1, case2)
        bot.send_message(call.message.chat.id, "*Выберите причину пересдачи:*\n\nПропуск по уважительной причине - есть подтверждающий документ.\nПропуск по неуважительной причине (нет подтверждающего документа) или набрано недостаточное количество баллов на экзамене/зачёте",reply_markup = markup_per, parse_mode='Markdown')

    elif call.data == 'Инфа':
        bot.send_message(call.message.chat.id, "Важная информация:")

    elif call.data == 'Контакты':
        bot.send_message(call.message.chat.id, "*Контакты работников студофиса:*\n\n_Павлова Юлия Олеговна_\n_Администратор_\nE-mail: yuopavlova@iate.obninsk.ru\nКабинет: 303-с\n\n_Павлова Нина Алексеевна_\n_Специалист по учебно-методической работе 1 категории_\nE-mail: napavlova@iate.obninsk.ru\nКабинет: С-303\n\n_Казанова Наталья Сергеевна_\n_Администратор_\nE-mail: nskazanova@iate.obninsk.ru\nКабинет: С-305\n\n_Лисуренко Ирина Петровна_\n_Администратор_\nE-mail: iplisurenko@iate.obninsk.ru\nКабинет: С-303\n\n_Чуенко Алла Борисовна_\n_Администратор (по вопросам материальной поддержки)_\nE-mail: abchuenko@iate.obninsk.ru\nКабинет: 1-112а", parse_mode= 'Markdown')
        bot.send_message(call.message.chat.id, "Чтобы вернуться к основному меню, введите /back")
    elif call.data == 'Онлайн':
        bot.send_message(call.message.chat.id, 'Место предоставления справки (наименование гос.органа/учреждения/организации)')
        bot.register_next_step_handler(call.message, get_spravka_online)

    elif call.data == 'Очно':
        bot.send_message(call.message.chat.id, 'Место предоставления справки (наименование гос.органа/учреждения/организации)')
        bot.register_next_step_handler(call.message, get_spravka_offline)
    
    elif call.data == 'Запись':
        bot.send_message(call.message.chat.id, "Укажите по какому вопросу Вам необходима консультация") 
        bot.register_next_step_handler(call.message, check_other_messages) 
        
        @bot.callback_query_handler( 
            func=lambda call: call.data.startswith(calendar_1_callback.prefix) 
        ) 
        def callback_inline(call: CallbackQuery): 
              # At this point, we are sure that this calendar is ours. So we cut the line by the separator of our calendar 
              name, action, year, month, day = call.data.split(calendar_1_callback.sep) 
              # Processing the calendar. Get either the date or None if the buttons are of a different type 
              date = calendar.calendar_query_handler( 
                  bot=bot, call=call, name=name, action=action, year=year, month=month, day=day 
              ) 
              # There are additional steps. Let's say if the date DAY is selected, you can execute your code. I sent a message. 
              if action == "DAY": 
                  bot.send_message( 
                      chat_id=call.from_user.id, 
                      text=f"Консультации проходят с 12:00 до 14:00, Вы выбрали дату {date.strftime('%d.%m.%Y')}", 
                      reply_markup=ReplyKeyboardRemove(), 
                  ) 
                  print(f"{calendar_1_callback}: Day: {date.strftime('%d.%m.%Y')} Консультации проходят с 12:00 до 14:00") 
                  # bot.send_message(call.from_user.id, 'Консультации проходят с 12:00 до 14:00')

                
 
              elif action == "CANCEL": 
                  bot.send_message( 
                      chat_id=call.from_user.id,
                      text="/back", 
                      reply_markup=ReplyKeyboardRemove(), 
                  ) 
                  print(f"{calendar_1_callback}: Cancellation") 
              

    elif call.data == 'Отмена':
        id = call.message.chat.id
        worksheet =  sh.worksheet('консультации')
        values_fio = worksheet.col_values(1) # 1 столбец - все ФИО
        if users[id] in values_fio:  # если человек записан
          cell = worksheet.find(users[id]) 
          worksheet.update_cell(cell.row,1, "")
          worksheet.update_cell(cell.row,2, "")
          worksheet.update_cell(cell.row,3, "")
          bot.send_message(call.message.chat.id, f'Ваша запись успешно отменена.')
          bot.send_message(call.message.chat.id, "Чтобы вернуться к основному меню, введите /back")

        else:
          bot.send_message(call.message.chat.id, f'Вы ещё не записаны.')
          bot.send_message(call.message.chat.id, f'Чтобы вернуться к основному меню, введите /back.')
    
    elif call.data == 'Уваж':
        bot.send_message(call.message.chat.id, "Прикрепите документ, подтверждающий отсутсвие\n*ВАЖНО* Формат pdf.\nО дате пересдачи вам ответит сотрудник позже", parse_mode = 'Markdown')
        bot.register_next_step_handler(call.message, spravka_uploaded)

    elif call.data == 'Неуваж':
        bot.send_message(call.message.chat.id, "В связи с тем, что причина неуважительная, считается, что экзамен сдан на оценку *неудовлетворительно*. Расписание пересдач будет позже", parse_mode = 'Markdown' )
        
        
bot.polling(none_stop=True, interval=0)